In [1]:
from etl import get_fred_data, fetch_world_bank_data, reindex_data, store_data_in_db, analyze_with_auto_arima, make_predictions, get_yahoo_data, combine_dataframes
import os
from fredapi import Fred
from sqlalchemy import create_engine
from dotenv import load_dotenv
import logging
import pandas as pd


logging.basicConfig(
    level=logging.DEBUG,
    format = "{asctime} - {levelname} - {message}",
    style = "{",
    datefmt = "%Y-%m-%d %H:%M",
)

load_dotenv()


fred_indicators = {
    "cpi": "CPIAUCSL",
    "federal_rate": "FEDFUNDS",
    "gdp": "GDP",
    "10y_treasury": "DGS10",
    "nasdaq_com": "NASDAQCOM",
}

fred_ins = Fred(api_key=os.environ.get("FRED_API", ""))
fred_data = get_fred_data(fred_ins, fred_indicators)

Key '-5419103323682258794' not in persistent cache.
Key '-3604272794836340258' not in persistent cache.
Key '-7839298695953911116' not in persistent cache.
Key '-8954656583958949463' not in persistent cache.
Key '2700295613665071291' not in persistent cache.
Key '5965127715663629597' not in persistent cache.
Key '-5403980666461440743' not in persistent cache.
Key '8807737506130540015' not in persistent cache.
Key '5117446250176770189' not in persistent cache.
Key '-6119937053031127332' not in persistent cache.
Key '-4698380380477061793' not in persistent cache.
Key '-1349712181199426397' not in persistent cache.
Key '-1600051166795501153' not in persistent cache.
Key '-703324260485636332' not in persistent cache.
Key '1919018347752383116' not in persistent cache.
Key '-4230284034572608888' not in persistent cache.
Key '-1551005725972495740' not in persistent cache.
Key '-7149973848489528339' not in persistent cache.
Key '-5077965500059468296' not in persistent cache.
Key '7769810451133

In [2]:
from etl import analyze_with_auto_arima, make_predictions

In [7]:
forecasts = []
results = {}
for df in fred_data:
    if df.empty:
        logging.warning(f"Skipping empty dataframe: {df.attrs.get('table_name', 'unknown')}")
        continue

    use_yearly_data = False
    table_name = df.attrs.get("table_name", "")

    # Use yearly data for World Bank data and FRED data except for 10y_treasury and nasdaq_com
    if ("world_bank" in table_name) or ("fred" in table_name and "10y_treasury" not in table_name and "nasdaq_com" not in table_name):
        use_yearly_data = True
        logging.info(f"Using yearly data for {table_name} predictions")

    # Analyze with auto_arima, using yearly data if appropriate
    results = {**results, **analyze_with_auto_arima(df, use_yearly_data=use_yearly_data)}
    if not results:
        logging.warning(f"No ARIMA results for {table_name}")
        continue
    # Make predictions, which will handle extrapolation from yearly to weekly if needed
    forecast = make_predictions(results, steps=10)  # 10 years of weekly data (52 weeks * 10 years)
    if forecast:
        forecasts.extend(forecast)
        logging.info(f"Successfully generated forecasts for {table_name}")
    else:
        logging.warning(f"No forecasts generated for {table_name}")

2025-07-17 21:42 - INFO - Using yearly data for fred_cpi predictions
2025-07-17 21:42 - INFO - Using 'cpi' column for ARIMA modeling
C:\Users\brzoza\Desktop\work\xeta\.venv\Lib\site-packages\pmdarima\arima\_validation.py:76: UserWarning: stepwise model cannot be fit in parallel (n_jobs=1). Falling back to stepwise parameter search.
  warnings.warn('stepwise model cannot be fit in parallel (n_jobs=%i). '
C:\Users\brzoza\Desktop\work\xeta\.venv\Lib\site-packages\sklearn\utils\deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
C:\Users\brzoza\Desktop\work\xeta\.venv\Lib\site-packages\sklearn\utils\deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
C:\Users\brzoza\Desktop\work\xeta\.venv\Lib\site-packages\sklearn\utils\deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6

Performing stepwise search to minimize aicc
 ARIMA(2,2,2)(0,0,0)[0]             : AICC=570.363, Time=0.08 sec
 ARIMA(0,2,0)(0,0,0)[0]             : AICC=657.565, Time=0.01 sec
 ARIMA(1,2,0)(0,0,0)[0]             : AICC=653.370, Time=0.01 sec
 ARIMA(0,2,1)(0,0,0)[0]             : AICC=626.530, Time=0.02 sec
 ARIMA(1,2,2)(0,0,0)[0]             : AICC=568.583, Time=0.04 sec
 ARIMA(0,2,2)(0,0,0)[0]             : AICC=567.964, Time=0.03 sec


C:\Users\brzoza\Desktop\work\xeta\.venv\Lib\site-packages\sklearn\utils\deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
C:\Users\brzoza\Desktop\work\xeta\.venv\Lib\site-packages\sklearn\utils\deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
C:\Users\brzoza\Desktop\work\xeta\.venv\Lib\site-packages\sklearn\utils\deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
2025-07-17 21:42 - INFO -   Best order: (0, 2, 2) (AIC: 567.89)
C:\Users\brzoza\Desktop\work\xeta\.venv\Lib\site-packages\sklearn\utils\deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
2025-07-17 21:42 - DEBUG - Step length for fred_cpi: 31
C:\Users\brzoza\Desktop\work

 ARIMA(0,2,3)(0,0,0)[0]             : AICC=568.325, Time=0.04 sec
 ARIMA(1,2,1)(0,0,0)[0]             : AICC=577.999, Time=0.03 sec
 ARIMA(1,2,3)(0,0,0)[0]             : AICC=570.037, Time=0.08 sec
 ARIMA(0,2,2)(0,0,0)[0] intercept   : AICC=569.893, Time=0.05 sec

Best model:  ARIMA(0,2,2)(0,0,0)[0]          
Total fit time: 0.390 seconds


2025-07-17 21:42 - INFO - Successfully generated forecasts for fred_cpi
2025-07-17 21:42 - INFO - Using yearly data for fred_federal_rate predictions
2025-07-17 21:42 - INFO - Using 'federal_rate' column for ARIMA modeling
C:\Users\brzoza\Desktop\work\xeta\.venv\Lib\site-packages\pmdarima\arima\_validation.py:76: UserWarning: stepwise model cannot be fit in parallel (n_jobs=1). Falling back to stepwise parameter search.
  warnings.warn('stepwise model cannot be fit in parallel (n_jobs=%i). '
C:\Users\brzoza\Desktop\work\xeta\.venv\Lib\site-packages\sklearn\utils\deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
C:\Users\brzoza\Desktop\work\xeta\.venv\Lib\site-packages\sklearn\utils\deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
C:\Users\brzoza\Desktop\work\xeta\.venv\Lib\site-packages\sklearn\utils\depre

...

 ARIMA(10,2,1)(0,0,0)[0]             : AICC=inf, Time=3.68 sec


C:\Users\brzoza\Desktop\work\xeta\.venv\Lib\site-packages\sklearn\utils\deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


 ARIMA(9,2,1)(0,0,0)[0]             : AICC=inf, Time=3.63 sec


C:\Users\brzoza\Desktop\work\xeta\.venv\Lib\site-packages\sklearn\utils\deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
2025-07-17 21:43 - INFO -   Best order: (10, 2, 0) (AIC: 78593.69)
C:\Users\brzoza\Desktop\work\xeta\.venv\Lib\site-packages\sklearn\utils\deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
2025-07-17 21:43 - DEBUG - Step length for fred_cpi: 31
C:\Users\brzoza\Desktop\work\xeta\.venv\Lib\site-packages\sklearn\utils\deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
2025-07-17 21:43 - DEBUG - Sample forecast for fred_cpi: 2025-07-01    322.515892
2025-08-01    323.303226
2025-09-01    324.090560
2025-10-01    324.877895
2025-11-01    325.665229
Freq: MS, dtype: float64
C:\Users\brzoza\Desktop\work\xe

 ARIMA(10,2,0)(0,0,0)[0] intercept   : AICC=78595.742, Time=0.82 sec

Best model:  ARIMA(10,2,0)(0,0,0)[0]          
Total fit time: 11.395 seconds


In [3]:
forecasts[3].index[2] - forecasts[3].index[1]

IndexError: list index out of range

In [5]:
results

{'fred_cpi': {'order': (0, 2, 2),
  'aic': 566.3638201678984,
  'model': ARIMA(order=(0, 2, 2), scoring_args={}, suppress_warnings=True,
        with_intercept=False),
  'is_yearly': True},
 'fred_federal_rate': {'order': (1, 1, 2),
  'aic': -386.3314706159667,
  'model': ARIMA(order=(1, 1, 2), scoring_args={}, suppress_warnings=True,
        with_intercept=False),
  'is_yearly': True},
 'fred_m2': {'order': (2, 2, 1),
  'aic': 3393.185230183458,
  'model': ARIMA(order=(2, 2, 1), scoring_args={}, suppress_warnings=True,
        with_intercept=False),
  'is_yearly': True},
 'fred_10y_treasury': {'order': (0, 1, 2),
  'aic': -18153.23521311859,
  'model': ARIMA(order=(0, 1, 2), scoring_args={}, suppress_warnings=True,
        with_intercept=False),
  'is_yearly': False},
 'fred_nasdaq_com': {'order': (10, 2, 0),
  'aic': 78547.5561012118,
  'model': ARIMA(order=(10, 2, 0), scoring_args={}, suppress_warnings=True,
        with_intercept=False),
  'is_yearly': False},
 'fred_corp_profits':

In [10]:
import numpy as np


final_forecast = {}
for name, result in results.items():
    s = 10
    model = result["model"]
    step_length = pd.DataFrame(model.predict(steps=2))
    diff = (step_length.index[2] - step_length.index[1])
    if isinstance(diff, pd.Timedelta):
        diff = diff.days
    if diff < 7:
        print('daily')
        forecast_values = pd.DataFrame(model.predict(n_periods=s * 365))
        indexresampled = np.linspace(forecast_values.index[0], forecast_values.index[-1], num=s * 52, endpoint=True)
        indexresampled = pd.Index(indexresampled)
        df_resampled = forecast_values.reindex(df.index.union(indexresampled)).interpolate().loc[indexresampled]
        final_forecast[name] = df_resampled
    elif 6 < diff < 8:
        print('weekly')
        forecast_values = pd.DataFrame(model.predict(n_periods=s * 52))
        final_forecast[name] = forecast_values
    elif 25 < diff < 35:
        print('monthly')
        forecast_values = pd.DataFrame(model.predict(n_periods=s * 12))
        forecast_values = forecast_values.resample("W").ffill()
        final_forecast[name] = forecast_values
    elif 35 < diff < 100:
        print('quarterly')
        forecast_values = pd.DataFrame(model.predict(n_periods=s * 4))
        forecast_values = forecast_values.resample("W").ffill()
        final_forecast[name] = forecast_values
    else:
        print('yearly')
        forecast_values = pd.DataFrame(model.predict(n_periods=s))
        forecast_values = forecast_values.resample("W").ffill()
        final_forecast[name] = forecast_values

monthly
monthly
quarterly
daily
daily


C:\Users\brzoza\Desktop\work\xeta\.venv\Lib\site-packages\sklearn\utils\deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
C:\Users\brzoza\Desktop\work\xeta\.venv\Lib\site-packages\sklearn\utils\deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
C:\Users\brzoza\Desktop\work\xeta\.venv\Lib\site-packages\sklearn\utils\deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
C:\Users\brzoza\Desktop\work\xeta\.venv\Lib\site-packages\sklearn\utils\deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
C:\Users\brzoza\Desktop\work\xeta\.venv\Lib\site-packages\sklearn\utils\deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure

In [14]:
final_forecast["fred_nasdaq_com"].shape

(520, 1)

In [78]:
schedule = pd.read_csv("./Xeta Estimated Token Distribution Schedule.csv")

In [79]:
schedule.columns = ["y", "m", "o token per month", "o acc token", "n token per month", "n acc token"]

In [80]:
schedule = schedule.drop(columns=["o token per month", "o acc token", "n acc token"])

In [81]:
schedule.shape

(120, 3)

In [12]:
cpi = pd.DataFrame(final_forecast["fred_cpi"])

In [27]:
final_forecast["fred_cpi"].shape

(518, 1)

In [62]:
prediction_data = pd.concat([final_forecast["fred_cpi"], pd.DataFrame([416.208670, 416.208670], index=['2035-06-10', '2035-06-17'])])
prediction_data.columns = ["cpi"]
prediction_data["y10_treasury"] = final_forecast["fred_10y_treasury"].values
prediction_data["nasdaq_com"] = final_forecast["fred_nasdaq_com"].values
prediction_data["gdp"] = pd.concat([final_forecast["fred_gdp"], pd.DataFrame([[]for _ in range(10)])], axis=1).ffill().values
prediction_data["federal_rate"] = pd.concat([final_forecast["fred_federal_rate"], pd.DataFrame([[]for _ in range(2)])], axis=1).ffill().values

In [63]:
prediction_data.index = pd.to_datetime(prediction_data.index)

In [64]:
prediction_data["token_schedule"] = 0

In [69]:
prediction_data

,cpi,y10_treasury,nasdaq_com,gdp,federal_rate,token_schedule
2025-07-06,322.515892,4.499930,20772.836560,30319.953010,4.322956,0
2025-07-13,322.515892,4.499922,21040.970521,30319.953010,4.322956,0
2025-07-20,322.515892,4.499915,21309.104482,30319.953010,4.322956,0
2025-07-27,322.515892,4.499908,21577.238444,30319.953010,4.322956,0
2025-08-03,323.303226,4.499901,21845.372405,30319.953010,4.313040,0
...,...,...,...,...,...,...
2035-05-20,415.421336,4.496206,158861.826665,44278.287408,4.221360,0
2035-05-27,415.421336,4.496199,159129.960626,44278.287408,4.221360,0
2035-06-03,416.208670,4.496192,159398.094588,44278.287408,4.221360,0
2035-06-10,416.208670,4.496185,159666.228549,44278.287408,4.221360,0


In [82]:
schedule = pd.to_numeric(schedule["n token per month"].str.replace(",",""))


In [83]:
schedule

0      6.062178e+08
1      2.511036e+08
2      1.926786e+08
3      1.624356e+08
4      1.431086e+08
           ...     
115    8.750000e+07
116    8.750000e+07
117    8.750000e+07
118    8.750000e+07
119    8.750000e+07
Name: n token per month, Length: 120, dtype: float64

In [84]:
c_month = 0
row_index = 0
for i, row in prediction_data.iterrows():
    if i.month != c_month:
        c_month = i.month
        prediction_data.at[i, "token_schedule"] = schedule.iloc[row_index]
        row_index += 1
    else:
        prediction_data.at[i, "token_schedule"] = 0

In [85]:
prediction_data

,cpi,y10_treasury,nasdaq_com,gdp,federal_rate,token_schedule
2025-07-06,322.515892,4.499930,20772.836560,30319.953010,4.322956,606217782.65
2025-07-13,322.515892,4.499922,21040.970521,30319.953010,4.322956,0
2025-07-20,322.515892,4.499915,21309.104482,30319.953010,4.322956,0
2025-07-27,322.515892,4.499908,21577.238444,30319.953010,4.322956,0
2025-08-03,323.303226,4.499901,21845.372405,30319.953010,4.313040,251103627.33
...,...,...,...,...,...,...
2035-05-20,415.421336,4.496206,158861.826665,44278.287408,4.221360,0
2035-05-27,415.421336,4.496199,159129.960626,44278.287408,4.221360,0
2035-06-03,416.208670,4.496192,159398.094588,44278.287408,4.221360,87500000.0
2035-06-10,416.208670,4.496185,159666.228549,44278.287408,4.221360,0


In [86]:
for c in prediction_data.columns:
    if c != "token_schedule":
        prediction_data[f"{c}_increase"] = prediction_data[c].pct_change()

In [87]:
prediction_data

,cpi,y10_treasury,nasdaq_com,gdp,federal_rate,token_schedule,cpi_increase,y10_treasury_increase,nasdaq_com_increase,gdp_increase,federal_rate_increase
2025-07-06,322.515892,4.499930,20772.836560,30319.953010,4.322956,606217782.65,NaN,NaN,NaN,NaN,NaN
2025-07-13,322.515892,4.499922,21040.970521,30319.953010,4.322956,0,0.000000e+00,-0.000002,0.012908,0.0,0.000000e+00
2025-07-20,322.515892,4.499915,21309.104482,30319.953010,4.322956,0,0.000000e+00,-0.000002,0.012743,0.0,0.000000e+00
2025-07-27,322.515892,4.499908,21577.238444,30319.953010,4.322956,0,0.000000e+00,-0.000002,0.012583,0.0,0.000000e+00
2025-08-03,323.303226,4.499901,21845.372405,30319.953010,4.313040,251103627.33,2.441226e-03,-0.000002,0.012427,0.0,-2.293839e-03
...,...,...,...,...,...,...,...,...,...,...,...
2035-05-20,415.421336,4.496206,158861.826665,44278.287408,4.221360,0,0.000000e+00,-0.000002,0.001691,0.0,0.000000e+00
2035-05-27,415.421336,4.496199,159129.960626,44278.287408,4.221360,0,0.000000e+00,-0.000002,0.001688,0.0,0.000000e+00
2035-06-03,416.208670,4.496192,159398.094588,44278.287408,4.221360,87500000.0,1.895267e-03,-0.000002,0.001685,0.0,-1.281685e-08
2035-06-10,416.208670,4.496185,159666.228549,44278.287408,4.221360,0,-1.084424e-09,-0.000002,0.001682,0.0,0.000000e+00


In [88]:
model_data = prediction_data.drop(["cpi", "y10_treasury", "nasdaq_com", "gdp", "federal_rate"], axis=1).fillna(0).to_csv("model_data.csv",index = False)

C:\Users\brzoza\AppData\Local\Temp\ipykernel_47804\1947718178.py:1: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  model_data = prediction_data.drop(["cpi", "y10_treasury", "nasdaq_com", "gdp", "federal_rate"], axis=1).fillna(0).to_csv("model_data.csv",index = False)
